In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.7 MB/s eta 0:00:00


In [4]:
import numpy as np
from datasets import load_dataset

In [6]:
dataset = load_dataset('cnn_dailymail', '1.0.0')

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [25]:
!pip install -U spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 34.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [44]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [94]:
noun_phrases = []

for i in range(0,1000):
  text = dataset['train'][i]['highlights']
  doc = nlp(text)
  noun_phrases.extend([f'Doc{i+1}: '+chunk.text for chunk in doc.noun_chunks])

print(noun_phrases)

['Doc1: Harry Potter star Daniel Radcliffe', 'Doc1: £20M fortune', 'Doc1: he', 'Doc1: Young actor', 'Doc1: he', 'Doc1: no plans', 'Doc1: his cash', "Doc1: Radcliffe's earnings", 'Doc1: first five Potter films', 'Doc1: trust fund', 'Doc2: Mentally ill inmates', 'Doc2: Miami', 'Doc2: the "forgotten floor" Judge Steven Leifman', 'Doc2: a result', 'Doc2: "avoidable felonies', 'Doc2: CNN', 'Doc2: facility', 'Doc2: patient', 'Doc2: I', 'Doc2: the son', 'Doc2: the president', 'Doc2: Leifman', 'Doc2: the system', 'Doc2: he', 'Doc2: change', 'Doc3: I', 'Doc3: I', 'Doc3: driver', 'Doc3: Man', 'Doc3: pickup truck', 'Doc3: half', 'Doc3: he', 'Doc3: face', 'Doc3: I', 'Doc3: a 30-, 35-foot free fall" Minnesota bridge', 'Doc3: rush hour', 'Doc4: procedure', 'Doc4: none', 'Doc4: spokesman', 'Doc4: President', 'Doc4: powers', 'Doc4: vice president', 'Doc4: Bush', 'Doc4: routine colonoscopy', 'Doc4: Camp David', 'Doc5: NFL chief', 'Doc5: Atlanta Falcons owner', "Doc5: Michael Vick's conduct", 'Doc5: NFL

In [95]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfidf = TfidfVectorizer()
context_matrix = tfidf.fit_transform(noun_phrases)

In [101]:
print(type(context_matrix))

<class 'scipy.sparse._csr.csr_matrix'>


In [104]:
tfidf.get_feature_names_out()

array(['000', '036', '047', ..., 'zuckerberg', 'zurich', 'zvonareva'],
      dtype=object)

In [98]:
def cosine_similarity_between_noun_phrases(np1, np2):
    np1_index = noun_phrases.index(np1)
    np2_index = noun_phrases.index(np2)
    cosine_sim = cosine_similarity(context_matrix[np1_index], context_matrix[np2_index])
    return cosine_sim[0][0]

noun_phrase_1 = "Doc1: Harry Potter star Daniel Radcliffe"
noun_phrase_2 = "Doc61: the Paris Masters"
similarity = cosine_similarity_between_noun_phrases(noun_phrase_1, noun_phrase_2)
print(f"Cosine similarity between '{noun_phrase_1}' and '{noun_phrase_2}': {similarity}")

noun_phrase_1 = "Doc1: Harry Potter star Daniel Radcliffe"
noun_phrase_2 = "Doc1: Radcliffe's earnings"
similarity = cosine_similarity_between_noun_phrases(noun_phrase_1, noun_phrase_2)
print(f"Cosine similarity between '{noun_phrase_1}' and '{noun_phrase_2}': {similarity}")

noun_phrase_1 = "Doc1: Harry Potter star Daniel Radcliffe"
noun_phrase_2 = "Doc1: £20M fortune"
similarity = cosine_similarity_between_noun_phrases(noun_phrase_1, noun_phrase_2)
print(f"Cosine similarity between '{noun_phrase_1}' and '{noun_phrase_2}': {similarity}")

Cosine similarity between 'Doc1: Harry Potter star Daniel Radcliffe' and 'Doc61: the Paris Masters': 0.0
Cosine similarity between 'Doc1: Harry Potter star Daniel Radcliffe' and 'Doc1: Radcliffe's earnings': 0.4553802533935939
Cosine similarity between 'Doc1: Harry Potter star Daniel Radcliffe' and 'Doc1: £20M fortune': 0.19760143682192866
